In [1]:
from pytential.sympy_pytential import sympy_pytential
from pytential.reduce import min_pytential

In [2]:
fa = sympy_pytential.from_properties({'mu0':[2,0]}, {'T':.300}, suffix='a')
fb = sympy_pytential.from_properties({'mu0':[0,2]}, {'T':.300}, suffix='b')
print(fa)

Density, rho, not found. Assuming rho = 1
Specific volumes, vi, not found. Assuming vi = 1/rho
Density, rho, not found. Assuming rho = 1
Specific volumes, vi, not found. Assuming vi = 1/rho

Variables
['c1a', 'c0a', 'Va']

Potential
    ⎛          ⎛   c0a   ⎞    ⎞                 ⎛   c1a   ⎞
c0a⋅⎜2.4942⋅log⎜─────────⎟ + 2⎟ + 2.4942⋅c1a⋅log⎜─────────⎟
    ⎝          ⎝c0a + c1a⎠    ⎠                 ⎝c0a + c1a⎠

Gradient
⎡          ⎛   c1a   ⎞            ⎛   c0a   ⎞       ⎤
⎢2.4942⋅log⎜─────────⎟, 2.4942⋅log⎜─────────⎟ + 2, 0⎥
⎣          ⎝c0a + c1a⎠            ⎝c0a + c1a⎠       ⎦

Hessian
⎡   2.4942⋅c0a        -2.4942       ⎤
⎢───────────────     ─────────     0⎥
⎢c1a⋅(c0a + c1a)     c0a + c1a      ⎥
⎢                                   ⎥
⎢    -2.4942         2.4942⋅c1a     ⎥
⎢   ─────────     ───────────────  0⎥
⎢   c0a + c1a     c0a⋅(c0a + c1a)   ⎥
⎢                                   ⎥
⎣       0                0         0⎦

Constraints
-Va + 1.0⋅c0a + 1.0⋅c1a



Make a collected function fa+fb with all the variables, and add constraints that the concentrations must sum to ca and cb

In [3]:
f = fa+fb
print(f)
f = sympy_pytential.add_constraints(f, ['c0', 'c1'])
print(f)


Variables
['c1a', 'c0b', 'Va', 'Vb', 'c0a', 'c1b']

Potential
    ⎛          ⎛   c0a   ⎞    ⎞                 ⎛   c0b   ⎞                 ⎛ 
c0a⋅⎜2.4942⋅log⎜─────────⎟ + 2⎟ + 2.4942⋅c0b⋅log⎜─────────⎟ + 2.4942⋅c1a⋅log⎜─
    ⎝          ⎝c0a + c1a⎠    ⎠                 ⎝c0b + c1b⎠                 ⎝c

  c1a   ⎞       ⎛          ⎛   c1b   ⎞    ⎞
────────⎟ + c1b⋅⎜2.4942⋅log⎜─────────⎟ + 2⎟
0a + c1a⎠       ⎝          ⎝c0b + c1b⎠    ⎠

Gradient
⎡          ⎛   c1a   ⎞            ⎛   c0b   ⎞                  ⎛   c0a   ⎞    
⎢2.4942⋅log⎜─────────⎟, 2.4942⋅log⎜─────────⎟, 0, 0, 2.4942⋅log⎜─────────⎟ + 2
⎣          ⎝c0a + c1a⎠            ⎝c0b + c1b⎠                  ⎝c0a + c1a⎠    

            ⎛   c1b   ⎞    ⎤
, 2.4942⋅log⎜─────────⎟ + 2⎥
            ⎝c0b + c1b⎠    ⎦

Hessian
⎡   2.4942⋅c0a                               -2.4942                     ⎤
⎢───────────────         0         0  0     ─────────            0       ⎥
⎢c1a⋅(c0a + c1a)                            c0a + c1a                    

In [4]:
f2 = min_pytential(f, vars_out=['c0', 'c1', 'Va', 'Vb'])

Visualization

In [6]:
 from numpy import linspace, vectorize, meshgrid
import plotly.graph_objects as go
from plotly.subplots import make_subplots

c0 = linspace(0.001, 0.999, 10)
va = linspace(0.001, 0.999, 10)
C0, Va = meshgrid(c0, va)

# Compute Z values using the fa function
F = f2(c0=C0, c1 = 1-C0, Va=Va, Vb=1-Va)

/usr/lib/python3/dist-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  warn('Singular Jacobian matrix. Using SVD decomposition to ' +
/usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/usr/lib/python3/dist-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  warn('Singular Jacobian matrix. Using SVD decomposition to ' +
/usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the

In [7]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'xy'}]])

# Add the 3D surface plot for f
fig.add_trace(go.Surface(z=F, x=C0, y=Va, colorscale='Viridis'), row=1, col=1)

# Add the 2D plot for f_reduced
fig.add_trace(go.Scatter(x=c0, y=fa(c0a=c0,c1a=1-c0,Va=1), mode='lines', name='fa'), row=1, col=2)
fig.add_trace(go.Scatter(x=c0, y=fb(c0b=c0,c1b=1-c0,Vb=1), mode='lines', name='fb'), row=1, col=2)
fig.add_trace(go.Scatter(x=c0, y=f2(c0=c0,c1=1-c0,Va=1, Vb=0), mode='lines', name='fb'), row=1, col=2)

# Update layout for the entire figure
fig.update_layout(
    title='3D Surface Plot of f and 2D Plot of f_reduced',
    scene=dict(
        xaxis_title='c0',
        yaxis_title='Va',
        zaxis_title='f',
        #xaxis2_title='c0',
        #yaxis2_title='f_reduced'
    ),

)

/usr/lib/python3/dist-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning:

Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.

/usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.

/usr/lib/python3/dist-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning:

Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.

/usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.

/usr/lib/python3/dist-packages/scipy/optimize/_t